In [1]:
from tetra.model import eval
from tetra.model import inference
from tetra.data import data_loader
from tetra.utils import file_io
from pathlib import Path
import importlib
import shutil
import os
from pycocotools.coco import COCO

In [2]:
importlib.reload(file_io)

<module 'tetra.utils.file_io' from '/Users/mateobigearddasen/Documents/agrosup_stage/Tetra_Project/src/tetra/utils/file_io.py'>

In [3]:
model = os.path.abspath("../../runs/train/lyrurus_yolov11l_new_dataset/weights/best.pt")
dataset = Path("../../dataset/og/images")
inference_dataset = os.path.abspath("../../dataset/inference/images")
gt_annotations = "../../dataset/og/_annotations.coco.json"

In [8]:
#Select images of the dataset
img_samples = data_loader.random_data_sample(dataset, 0.025, [".jpg"])

In [9]:
#Change images directory
for img in img_samples:
    img = Path(img)
    image = dataset / img
    shutil.move(image, Path(inference_dataset) / img.name)

In [10]:
#Get ground truth annotations
coco_gt = COCO(gt_annotations)
#Map id to image name
file_to_id = {img["file_name"]: img["id"] for img in coco_gt.dataset["images"] if img["file_name"] in img_samples}

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [ ]:
#Inference of the data 
predictions = []
for f in Path(inference_dataset).iterdir():
    if f.suffix.lower() == ".jpg":
        prediction = inference.solo_inference(model, inference_dataset + f"/{f.name}")
        prediction = prediction.to_coco_annotations()
        for ann in prediction:
            #Convert id to the gt id
            ann["image_id"] = file_to_id[f.name]
            ann["category_id"] = 1
            #Change category name
            ann["category_name"] = "Lyrurus_tetrix"
        predictions.append(prediction)

Performing prediction on 63 slices.


In [ ]:
metric_val = eval.eval_hr_images("../../dataset/og/_annotations.coco.json", "./annotations/yolov11l_new_dataset.coco.json")

In [ ]:
#Filter predictions
filtered_predictions = [ann for ann in file_io.flatten_pred(predictions) if ann["score"] > 0.386]

In [ ]:
#Save predictions
file_io.save_coco_labels(filtered_predictions, "./annotations", "yolov11l_new_dataset.coco.json" )

In [ ]:
metric_val.evaluate()
metric_val.accumulate()
metric_val.summarize()